In [47]:
import boto3
import json
from jinja2 import Environment, FileSystemLoader
import sys
from pathlib import Path
import os

In [48]:
template_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts-templates"))
output_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","model-results/cleaned-appraisal-report-rdf/"))
ontology_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","ontologies"))
ontology_name = "PropertyAppraisalOntology-v2.ttl"

input_text_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts"))
input_text_name = "Appraisal-Report-minimum-info-page-divisors.txt"

template_filename = "rdf-graph-template.txt"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

environment = Environment(loader=FileSystemLoader(template_dir))
template = environment.get_template(template_filename)

with open(os.path.join(ontology_dir,ontology_name),"r") as f:
    ontology = f.read()
with open(os.path.join(input_text_dir,input_text_name),"r") as f:
    input_text = f.read()

In [49]:
bedrock_runtime= boto3.client("bedrock-runtime",region_name="us-east-1")

In [50]:
report_pages ={t.split("\n")[0].strip().replace(" ","-"):t.split(t.split("\n")[0])[1] for t in input_text.split("-----")[1:]}

In [57]:
# for p_name,content in report_pages.items():
#     prompt = template.render(
#         ontology=ontology,
#         input_text=content
#     )
#     message={"role": "user","content": prompt}
#     messages = [message]
#     print(prompt)
#     body=json.dumps(
#     {
#         'messages':messages,
#         'anthropic_version':'bedrock-2023-05-31',
#         'top_p':0.5,
#         'temperature':0.5,
#         'top_k':5,
#         'max_tokens':3000
#     })
#     response = bedrock_runtime.invoke_model(
#         modelId=model_id,
#         contentType='application/json',
#         accept='application/json',
#         body=body
#     )
#     response_body = json.loads(response['body'].read())
#     break

In [59]:
prompt = template.render(
    ontology=ontology,
    input_text=report_pages["PAGE-2"]
)
message={"role": "user","content": prompt}
messages = [message]
print(prompt)
body=json.dumps(
{
    'messages':messages,
    'anthropic_version':'bedrock-2023-05-31',
    'top_p':0.5,
    'temperature':0.5,
    'top_k':5,
    'max_tokens':3000
})
response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType='application/json',
    accept='application/json',
    body=body
)
response_body = json.loads(response['body'].read())

Using this provided ontology, between <ontology> tags, exclusively, please create specific instances and data about the property appraisal from the following input text, between <input_text> tags. Also, create a full and complete RDF graph without leaving any statements from the graph.

<ontology>
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix mf: <http://example.org/mf> .

<mf:NamedEntity> a xsd:string .
<mf:Property> a <mf:NamedEntity> .
<mf:SubjectProperty> a <mf:Property> .
<mf:PropertyAddress> a <mf:NamedEntity> .
<mf:PropertyStreetNumber> a <mf:NamedEntity> .
<mf:PropertyStreetName> a <mf:NamedEntity> .
<mf:PropertyCity> a <mf:NamedEntity> .
<mf:PropertyZip> a <mf:NamedEntity> .
<mf:PropertyName> a <mf:NamedEntity> .
<mf:PropertyUnits> a <mf:NamedEntity> .
<mf:PropertyAppraisedValue> a <mf:NamedEntity> .
<mf:PropertyOwner> a <mf:NamedEntity> .
<mf:Prope

In [60]:
response_body

{'id': 'msg_019C2BhTHKusvyZjucSrwDrZ',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': 'Here is the RDF graph created from the provided input text, using the given ontology:\n\n<ontology>\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n@prefix mf: <http://example.org/mf> .\n\n<mf:Property_1> a <mf:Property> ;\n    <mf:hasName> "College Courtyard Apartments & Raider Housing" ;\n    <mf:hasAddress> <mf:PropertyAddress_1> ;\n    <mf:hasUnits> "62"^^xsd:int ;\n    <mf:hasOwner> <mf:PropertyOwner_1> ;\n    <mf:hasAppraisal> <mf:PropertyAppraisal_1> .\n\n<mf:PropertyAddress_1> a <mf:PropertyAddress> ;\n    <mf:hasStreetNumber> "28" , "30" ;\n    <mf:hasStreetName> "Garden Lane" ;\n    <mf:hasCity> "Niceville" ;\n    <mf:hasState> "FL" ;\n    <mf:hasZip> "32578" .\n\n<mf:PropertyOwner_1> a <mf:PropertyOwner> ;\n    rdfs:label "Owner" ;\

In [62]:
p_name="PAGE-2"

In [63]:
full_output_name = p_name
full_output_name=full_output_name.replace(":","-")
same_name_count = int(len([f for f in os.listdir(output_dir) if full_output_name in f])/2)
file_count = "" if same_name_count ==0 else f'-{same_name_count}'

In [64]:
full_output_name

'PAGE-2'

In [65]:
with open(os.path.join(output_dir,f"{full_output_name}{file_count}.json"),"w+") as f:
    json.dump(response_body,f)